## 6.1 构建风控模型流程

在对数据进行缺失值填补、异常值检测，以及通过可视化查看各特征分布以及特征之间关系之后，我们基于信用指标体系，开始构建风控模型。

构建模型的流程为：

> 1.按照 8:2 将客户数据集随机划分为训练集与测试集；
> 
> 2.使用逻辑回归和随机森林两种算法，分别构建风控模型；
> 
> 3.使用正则化、网格搜索、交叉验证等方式对模型参数进行调优。


![](https://ai-studio-static-online.cdn.bcebos.com/78882a201f434c8499cd27a2c33d89165c2cc5b12d8941f9801647354952d1b3)

                                                            构建风控模型流程

## 6.2 分层采样简介

在数据探索阶段，我们已经对预测特征Default的取值分布进行了分析，如下图所示。

![](https://ai-studio-static-online.cdn.bcebos.com/04a31d429be04445b214f4df2354f33652fdcaf743684a90b8cab7d76753cd2f)


在本项目中，需要通过预测客户是否违约来构建风控模型。数据集中违约客户远少于未违约客户，为了更客观地构建风控模型和评估模型效果，应该尽量使得训练集和测试集中违约客户的比例相一致。因此需要使用分层采样的方法来划分训练集和测试集。训练集和测试集划分的过程分为三步：分层、随机划分和合并。划分示意图如下：

![](https://ai-studio-static-online.cdn.bcebos.com/255a0a7b89d84a5aaf7ef5ef469d579481e4bf1d4e7d4ce6bf4fac3f6b0bbb73)

                                                       分层采样示例

可见，经过分层采样的划分后，训练集和测试集中违约样本的比例会保持一致。

## 6.3 训练集与测试集划分

下面我们开始对数据进行训练集与测试集的划分。在 Sklearn 中的 `model_selection` 模块，存在 `train_test_split()` 函数，用作训练集和测试集划分，函数语法为：`train_test_split(x,y,test_size = None,random_state = None,stratify = y)`，其中：
- `x,y`: 分别为预测所需的所有特征，以及需要预测的特征（即违约情况 Default）。
- `test_size`: 测试集比例，例如 `test_size=0.2` 则表示划分 20% 的数据作为测试集。
- `random_state`: 随机种子，因为划分过程是随机的，为了进行可重复的训练，需要固定一个 random_state ，结果重现。
- `stratify`: 使用分层采样，保证从违约客户和未违约客户中抽取了同样比例的训练集和测试集。
- 函数最终将返回四个变量，分别为 x 的测试集和训练集，以及 y 的测试集和训练集。


## 实训任务

- data 数据已经进行了分离为 `x, y`，请使用 `train_test_split` 函数进行训练集测试集划分。
- 其中测试集比例为 `0.2`。
- 随机种子 `random_state` 设定为 `33`。
- 分层参数 `stratify` 设置为 `y` (即按 `Default` 进行分层)。
- 返回的变量命名为` x_train, x_test, y_train, y_test`，类型为 `ndarray` 数组对象。


经过划分，我们已经将 data 划分为训练模型以及测试模型。同时由于我们采用了分层采样，保证了训练集和测试集中未违约客户人数与违约客户人数比例均衡。接下来我们将训练集带入算法中进行模型训练。

In [2]:
import pandas as pd
data = pd.read_table('dataset11(不良记录).txt',sep='\t')
from sklearn.model_selection import train_test_split

# 筛选data中的Default列的值，赋予变量y
y = data['Default'].values

# 筛选除去Default列的其他列的值，赋予变量x
x = data.drop(['Default'], axis=1).values

# 使用train_test_split方法，将x,y划分训练集和测试集
x_train, x_test, y_train, y_test = x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=33, stratify=y)

# 查看划分后的x_train与x_test的长度
len_x_train = len(x_train)
len_x_test = len(x_test)
print('x_train length: %d, x_test length: %d'%(len_x_train,len_x_test))

# 查看分层采样后的训练集中违约客户人数的占比
train_ratio = y_train.sum()/len(y_train)
print(train_ratio)

# 查看分层采样后的测试集中违约客户人数的占比
test_ratio = y_test.sum()/len(y_test)
print(test_ratio)


x_train length: 37865, x_test length: 9467
0.08995114221576654
0.08989120101404881


## 6.4 使用逻辑回归建立风险评估模型 

# 逻辑回归简介：
我们知道，在线性回归中预测目标 `y` 是连续型，且可以写成样本 `x` 每一个特征的线性加权形式：
$$y = w_1 x_1 + w_2 x_2 + \cdots +w_d x_d + w_0 = \boldsymbol{w}^T \boldsymbol{x}$$
其中，$d$ 为特征维度，参数向量 $\boldsymbol{w}\in \mathcal{R}^{d+1},以及样本向量 \boldsymbol{x}\in \mathcal{R}^{d+1}$ 都进行了数据增广。 现在，假设我们解决的是二分类问题。为了能够利用回归的方法来解决二分类问题，我们需要对线性回归的输出作改变。 逻辑回归采用的方法是引入一个 Logistic 函数，将连续型的输出映射到 (0,1) 之间。 Logistic 函数的定义如下： 
$$\sigma(x) = \frac{1}{1+e^{-x}}$$

对应的函数曲线如下图所示， 当输入 x 很大或很小时，该函数以接近于 0 或 1 的值输出，且 $\sigma(0)=0.5$。


![](https://ai-studio-static-online.cdn.bcebos.com/e9290c84cc7741619af0ab7fc4c0975d638d245c823347b9adeafb18cf925163)


有了 Logistic 函数 $\sigma(x)$，我们可以将任意实数映射到 (0,1) 之间。 此时样本属于正类 (y=1)的概率可以表示为： 
$$p(y=1|\boldsymbol{x})=\frac{1}{1+e^{-\boldsymbol{w}^T\boldsymbol{x}}}$$

逻辑回归在线性回归的基础上，利用一个非线性函数，建立了二元预测目标与原始输入之间的关系。

下面我们通过逻辑回归构建风险评估模型。

在 sklearn 的 `linear_model` 模块中，可以直接调用 `LogisticRegression`。再将划分好的训练集 `x_trian，y_train` 带入模型中进行训练。
模型训练好后便可带入测试集 `x_test` 进行预测。预测通常有两种方式：`predict` 和 `predict_proba`，前者直接预测出二进制分类标签（0或1），后者则预测出目标分数为正类(1)或负类(0)的概率估计(取值在0-1之间)。


## 实训任务

- 使用 `LogisticRegression` 新建一个模型对象，命名为 `lr`。
- 对lr对象调用 `fit` 方法，带入训练集 `x_train`， `y_train` 进行训练。
- 对训练好的 `lr` 模型调用 `predict` 方法，带入测试集 `x_test` 进行预测，将结果保存到变量 `y_predict` 中。


通过以上步骤，我们已经训练好了一个简单的逻辑回归模型，并在测试集上做了相应的预测，接下来需要对模型效果进行评估，我们使用 AUC 值评估模型效果。 

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import joblib
import pandas as pd
data = pd.read_table('dataset11(不良记录).txt',sep='\t')
import warnings
warnings.filterwarnings("ignore")

# 划分训练集和测试集
y = data['Default'].values
x = data.drop(['Default'], axis=1).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


# 调用模型，新建模型对象
lr = LogisticRegression()

# 带入训练集x_train, y_train进行训练
lr.fit(x_train, y_train)

# 保存训练好的模型：lr 是一个 LogisticRegression 模型
joblib.dump(lr, 'lr_yours.model')

# 对训练好的lr模型调用predict方法,带入测试集x_test进行预测
y_predict = lr.predict(x_test)

# 查看模型预测结果
print(y_predict[:10])
print(len(y_predict))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
9467


#### 如何评价模型性能呢
y_predict 存储了模型预测出的样本类别标签，y 中存储了样本的真实类别标签，可以评估二者的一致性作为性能评估的一种方式，比如统计一下两个向量的差值向量中非零元素的数量即为错误判定的样本数量，除以总样本数量，可以计算出错误率。试用代码实现错误率的计算。


In [4]:
import numpy as np

# 假设 y_test 和 y_predict 已经定义
# y_test 是测试集的真实类别标签
# y_predict 是模型预测出的测试集样本类别标签

# 确保 y_test 和 y_predict 长度一致
assert len(y_test) == len(y_predict), "y_test 和 y_predict 的长度不一致"

# 计算错误率
error_rate = np.sum(y_test != y_predict) / len(y_test)

print(f'错误率: {error_rate:.2f}')

错误率: 0.09


#### 问题：错误率作为性能指标能在类别不平衡问题下存在什么不足？

错误率是一个简单的度量，等于错误预测的数量除以总预测的数量。类别不平衡的数据集中，一个类别的样本数量远远多于另一个类别，它可能会受到这种不平衡的影响,因为错误率不考虑类别分布，所以它可能会高估对少数类别的分类性能。

## 6.5 逻辑回归风控模型效果评估

模型建立好之后，需要对模型的效果进行评估，此处我们选择使用 AUC 值：

对于二分类问题，一个样本的类别只有两种，我们用 0,1 分别表示两种类别。 当我们用一个分类器进行预测的时候，对于真实为 0 的样本，我们可能预测其为 0 或 1，同样对于真实为 1 的样本，我们也可能预测其为 0 或 1，这样就有四种可能性：

![](https://ai-studio-static-online.cdn.bcebos.com/bcde1b8a36a043fb8b03f94313815e10f65bd20242664385b59e88027dd588da)


真正率(True positive rate， TPR)，预测为正且实际为正的样本占所有正例样本的比例，可以更加直观地理解为“**正例样本的识别正确率**”。

假正率(False positive rate， FPR)，预测为正但实际为负的样本占所有负例样本的比例，可以更加直观地理解为“**负例样本的识别错误率**”。

真正率(TPR)和假正率(FPR)的定义为 

![](https://ai-studio-static-online.cdn.bcebos.com/9dad209b883a468d879d0f941d25564f4e6e34c7a3eb43ba82e2f521b9173c66)

利用 FPR 和 TPR，我们可以画出 ROC 曲线，ROC 曲线是基于样本的真实类别和预测概率。 


![](https://ai-studio-static-online.cdn.bcebos.com/6ce1d886ab024f68b86fdbeba0e976f3e5e6f03eba4f4e5989c87de6a80ea7e7)

横坐标：假正率( FPR)；纵坐标：真正率( TPR)。

在一个二分类模型中，假设采用逻辑回归分类器，其给出针对每个实例为正类的概率，那么通过设定一个阈值如 0.6，概率大于等于 0.6 的为正类，小于 0.6 的为负类。对应的就可以算出一组(FPR,TPR)，在平面中得到对应坐标点。随着阈值的逐渐减小，越来越多的实例被划分为正类，但是这些正类中同样也掺杂着真正的负实例，即 TPR 和 FPR 会同时增大。

AUC 则代表 ROC 曲线下方区域的面积，用于二分类模型的评价。全称为“Area under curve”，AUC 的数值范围是 [0.5,1]，值越大说明模型相对越好。
计算 AUC 最便捷的方式是使用 `Sklearn` 库中 `metrics` 模块的 `roc_auc_score()` 函数，函数语法为：`roc_auc_score（y_true，y_score...)`，其中：

- `y_true` -- 真正的二进制标签。
- `y_score` -- 目标分数为正类的概率估计。
- 函数最终返回的是 AUC 值。


## 实训任务

- 对训练好的 lr 模型调用 `predict_proba` 方法，带入测试集 `x_test` 进行预测,将结果保存到变量 `y_predict_proba` 中。
- `predict_proba` 方法返回的是目标分数负类 (0) 和正类 (1) 的概率估计，区间为 [0,1]；此外返回形式为一个二维的数组对象，第一个索引是指数据属于 0 类的概率，第二个索引是指数据属于 1 类的概率，他们相加和为 1。我们需要从 `y_predict_proba` 中取正类 (1) 的概率,将结果保存到变量 `y_predict` 中。
- 调用 `roc_auc_score()` 函数，将 `y_test`，`y_predict` 作为输入参数，求出测试集 AUC 值，将结果赋予 `test_auc`。

现在我们已经算出 AUC 值，接下来我们尝试调整参数来优化这个模型。 

In [6]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import joblib
import pandas as pd
data = pd.read_table('dataset11(不良记录).txt',sep='\t')
y = data['Default'].values
x = data.drop(['Default'], axis=1).values
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=33,stratify=y)

#调用上一小节中训练好的逻辑回归模型
lr = joblib.load("lr_yours.model")


# 对训练好的 lr 模型调用 predict_proba 方法，带入测试集 x_test 进行预测
y_predict_proba = lr.predict_proba(x_test)


# 查看概率估计前十行
print(y_predict_proba[:10])

# 取目标分数为正类(1)的概率估计
y_predict = y_predict_proba[:, 1]

# 利用roc_auc_score查看模型效果
test_auc = roc_auc_score(y_test, y_predict)

print('逻辑回归模型 test_auc:',test_auc)

[[7.23495635e-01 2.76504365e-01]
 [5.64931362e-01 4.35068638e-01]
 [8.48322243e-01 1.51677757e-01]
 [1.00000000e+00 1.10381248e-19]
 [9.99928090e-01 7.19104139e-05]
 [9.99993738e-01 6.26199754e-06]
 [5.67488952e-01 4.32511048e-01]
 [9.97775534e-01 2.22446607e-03]
 [5.01262712e-01 4.98737288e-01]
 [9.99998584e-01 1.41631134e-06]]
逻辑回归模型 test_auc: 0.6490175684949816


#### 问题思考：
> 1、ROC曲线是如何绘制的？

> 2、ROC曲线横轴上从左到右，所对应的阈值大小的次序是什么？从大到小 or 从小到大？

> 3.给定阈值基于simoid的输出值进行类别判定时，为什么大于该阈值的判定为正例？

请在此作答：1.ROC曲线是在不同分类阈值下，真正例率与假正例率之间的权衡关系，对于每个阈值，我们在坐标系中标记一个点，其横坐标是假正例率，纵坐标是真正例率；
2.横轴上的阈值次序是从大到小。也就是说，从左到右，阈值逐渐减小。这是因为通常从高阈值开始，然后逐步降低阈值，观察模型的性能变化；
3.sigmoid函数将负无穷到正无穷的值映射到[0, 1]之间，且随着输入值增大，输出趋近于1，大于阈值的输出对应的概率更接近1，我们将其判定为正例。